In [1]:
from datasets import load_from_disk

import numpy as np

from minbpe import BasicTokenizer

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import  Dataset, DataLoader

## Tokenizer

In [2]:
tokenizer = BasicTokenizer()
tokenizer.load("model/model_article_1000.model")

In [3]:
train_tokenized = load_from_disk("data/cnn_train_tokenized_10k")
val_tokenized = load_from_disk("data/cnn_val_tokenized_2500")
test_tokenized = load_from_disk("data/cnn_test_tokenized_2500")

## Vectorizer

In [4]:
class Vectorizer:
  def __init__(self, tokenizer: BasicTokenizer):
    self.tokenizer = tokenizer
    self.vocab_size = len(tokenizer.vocab)
    self.sos_idx = tokenizer.special_tokens["<sos>"]
    self.eos_idx = tokenizer.special_tokens["<eos>"]
    self.pad_idx = tokenizer.special_tokens["<pad>"]

  def index_vectorize(self, tokens, max_length=1024):
    indices = tokens[:max_length - 2]
    indices = [self.sos_idx] + indices + [self.eos_idx]
    indices += [self.pad_idx] * (max_length - len(indices))
    return indices
  
article_vectorizer = Vectorizer(tokenizer)

## Dataset

In [5]:
class IndexArticleDataset(Dataset):
  def __init__(self, input, target, vectorizer: Vectorizer, max_input_length=1024, max_target_length=128):
    self.input = input
    self.target = target
    self.vectorizer = vectorizer
    
    self.max_input_length = max_input_length
    self.max_target_length = max_target_length
    
    self.sos_index = vectorizer.sos_idx
    self.eos_index = vectorizer.eos_idx
    self.pad_index = vectorizer.pad_idx

    # Precompute indexed and padded input/target sequences
    self.indexed_input = [
      torch.as_tensor(
        vectorizer.index_vectorize(example, max_length=max_input_length), dtype=torch.long
      )
      for example in input
    ]
    self.indexed_target = [
      torch.as_tensor(
        vectorizer.index_vectorize(example, max_length=max_target_length), dtype=torch.long
      )
      for example in target
    ]

  def __len__(self):
    return len(self.input)

  def __getitem__(self, index):
    return {
            'x': self.indexed_input[index],
            'y': self.indexed_target[index]
        }
    # return {'x': torch.as_tensor(self.vectorizer.index_vectorize(self.input[index], self.max_input_length)),
    #         'y': torch.as_tensor(self.vectorizer.index_vectorize(self.target[index], self.max_target_length))}

  def get_vectorizer(self):
    return self.vectorizer 
  
  def get_num_batches(self, batch_size):
    return len(self) // batch_size 

In [6]:
train_dataset = IndexArticleDataset(train_tokenized['article'], 
                                    train_tokenized['highlights'], 
                                    article_vectorizer,
                                    max_input_length=512,
                                    max_target_length=128)

val_dataset = IndexArticleDataset(val_tokenized['article'], 
                                  val_tokenized['highlights'], 
                                  article_vectorizer,
                                  max_input_length=512,
                                  max_target_length=128)

test_dataset = IndexArticleDataset(test_tokenized['article'], 
                                   test_tokenized['highlights'], 
                                   article_vectorizer,
                                   max_input_length=512,
                                   max_target_length=128)

## Dataloader

In [7]:
def generate_batches(dataset, batch_size, 
                     shuffle=True,
                     drop_last=True, 
                     device="cpu"):
  
  dataloader = DataLoader(dataset=dataset, 
                          batch_size=batch_size,
                          shuffle=shuffle, 
                          drop_last=drop_last)

  for data_dict in dataloader:
    out_data_dict = {}
    for name, tensor in data_dict.items():
      out_data_dict[name] = data_dict[name].to(device)
    yield out_data_dict

## Model

In [25]:
class Head(nn.Module):
  def __init__(self, embed_dim, head_dim, max_input_length, dropout=0.5, is_masked=True):
    super().__init__()
    self.query = nn.Linear(embed_dim, head_dim, bias=False)
    self.key = nn.Linear(embed_dim, head_dim, bias=False)
    self.value = nn.Linear(embed_dim, head_dim, bias=False)
    self.is_masked = is_masked
    if is_masked:
      self.register_buffer('mask', torch.tril(torch.ones(max_input_length, max_input_length)))
    self.dropout = nn.Dropout(dropout)
  
  def forward(self, x, context=None):
    B, T, C = x.shape # (batch_size, time_steps, channels=embed_dim)

    if context == None:
      context = x

    Q = self.query(x)
    K = self.key(context)

    KQ = (Q @ K.transpose(-2, -1)) 
    KQ_scaled = KQ / (np.sqrt(K.shape[-1]))
    if self.is_masked:
      KQ_scaled = KQ_scaled.masked_fill(self.mask[:T, :T] == 0, float('-inf'))
    KQ_normalized_masked_softmaxed = torch.softmax(KQ_scaled, dim=-1)
    KQ_normalized_masked_softmaxed_dropouted = self.dropout(KQ_normalized_masked_softmaxed)

    V = self.value(context)
    out = KQ_normalized_masked_softmaxed_dropouted @ V
    return out
  
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, embed_dim, max_input_length, dropout=0.5, is_masked=True):
    super().__init__()
    head_dim = embed_dim // num_heads
    self.heads = nn.ModuleList([Head(embed_dim, head_dim, max_input_length, dropout, is_masked) for _ in range(num_heads)])
    self.linear = nn.Linear(head_dim * num_heads, embed_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, k, context=None):
    out = torch.cat([h(k, context) for h in self.heads], dim=-1)
    out = self.linear(out)
    out = self.dropout(out)
    return out
  
class FeedFoward(nn.Module):
  def __init__(self, embed_dim, dropout=0.5):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(embed_dim, 4*embed_dim), # default: embed_dim=512, feed_forward_dim=2048)
      nn.ReLU(),
      nn.Linear(4*embed_dim, embed_dim),
      nn.Dropout(dropout),
    )

  def forward(self, x, context=None):
    return self.layers(x)
  
class SubLayer(nn.Module):
  def __init__(self, layer: nn.Module, embed_dim, dropout=0.5):
    super().__init__()
    self.Layer = layer
    self.ln = nn.LayerNorm(embed_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, context=None):
    out = self.Layer(x, context)
    out = self.dropout(out)
    out = self.ln(x + out) # normalization after residual connection
    return out
  
class Encoder(nn.Module):
  def __init__(self, num_heads, embed_dim, max_input_length, dropout=0.5):
    super().__init__()
    self.multi_head_attention = SubLayer(MultiHeadAttention(num_heads, embed_dim, max_input_length, dropout, is_masked=False), embed_dim, dropout)
    self.feed_forward = SubLayer(FeedFoward(embed_dim, dropout), embed_dim, dropout)

  def forward(self, x):
    out = self.multi_head_attention(x)
    out = self.feed_forward(out)
    return out
  
class Decoder(nn.Module):
  def __init__(self, num_heads, embed_dim, max_input_length, dropout=0.5):
    super().__init__()
    self.masked_multi_head_attention = SubLayer(MultiHeadAttention(num_heads, embed_dim, max_input_length, dropout, is_masked=True), embed_dim, dropout)
    self.unmasked_multi_head_attention = SubLayer(MultiHeadAttention(num_heads, embed_dim, max_input_length, dropout, is_masked=False), embed_dim, dropout)
    self.feed_forward = SubLayer(FeedFoward(embed_dim, dropout), embed_dim, dropout)

  def forward(self, x, context=None):
    out = self.masked_multi_head_attention(x)
    
    # In Encoder-Decoder model, context is Encoder output
    # In Decoder-only model, context is None and skip this process
    if context is not None:
      out = self.unmasked_multi_head_attention(out, context)
    
    out = self.feed_forward(out)
    return out

class PositionalEncoding(nn.Module):
  def __init__(self, embed_dim, max_input_length):
    super().__init__()

    # (max_input_length, 1)
    pos = torch.arange(0, max_input_length, dtype=torch.float).unsqueeze(1)
    
    # (embed_dim/2,)
    i = torch.arange(0, embed_dim, 2).float()
    div_term = torch.exp(i * (-np.log(10000.0) / embed_dim))

    # (max_input_length, embed_dim/2)
    rate = pos * div_term

    pe = torch.zeros(max_input_length, embed_dim)
    pe[:, 0::2] = torch.sin(rate)  # even indices
    pe[:, 1::2] = torch.cos(rate)  # odd indices

    pe = pe.unsqueeze(0)  # (1, T, D) - for broadcasting over batch
    self.register_buffer('pe', pe)

  def forward(self, x):
    B, T = x.shape
    return self.pe[:, :T, :]
  
# Decoder only model
class Transformer_Decoder(nn.Module):
  def __init__(self, num_layers, vocab_size, num_heads, embed_dim, max_input_length, dropout=0.5):
    super().__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim)
    self.pos_encoder = PositionalEncoding(embed_dim, max_input_length)
    self.dropout = nn.Dropout(dropout)
    self.layers = nn.Sequential(
      *[Decoder(num_heads, embed_dim, max_input_length, dropout) for _ in range(num_layers)],
      nn.Linear(embed_dim, vocab_size),
      )

  def forward(self, x):
    embed_x = self.embed(x)
    pos_embed_x = self.pos_encoder(x)
    embed_input = embed_x + pos_embed_x
    embed_input = self.dropout(embed_input)
    out = self.layers(embed_input)
    return out

# Encoder-Decoder model
class Transformer_Encoder_Decoder(nn.Module):
  def __init__(self, vocab_size, num_layers, num_heads, embed_dim, max_input_length, max_target_length, pad_idx, dropout=0.5):
    super().__init__()
    self.input_embed = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
    self.input_pos_encoder = PositionalEncoding(embed_dim, max_input_length)
    self.input_dropout = nn.Dropout(dropout)
    
    self.target_embed = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
    self.target_pos_encoder = PositionalEncoding(embed_dim, max_target_length)
    self.target_dropout = nn.Dropout(dropout)
    
    self.encoder = nn.ModuleList([Encoder(num_heads, embed_dim, max_input_length, dropout) for _ in range(num_layers)])
    self.decoder = nn.ModuleList([Decoder(num_heads, embed_dim, max_input_length, dropout) for _ in range(num_layers)])
    self.linear = nn.Linear(embed_dim, vocab_size)

  def forward(self, x, y):
    embed_x = self.input_embed(x)
    pos_embed_x = self.input_pos_encoder(x)
    embed_input = embed_x + pos_embed_x
    embed_input = self.input_dropout(embed_input)
    
    embed_y = self.target_embed(y)
    pos_embed_y = self.target_pos_encoder(y)
    embed_target = embed_y + pos_embed_y
    embed_target = self.target_dropout(embed_target)

    for layer in self.encoder:
      embed_input = layer(embed_input)
    encoder_out = embed_input

    for layer in self.decoder:
      embed_target = layer(embed_target, encoder_out)
    decoder_out = embed_target

    out = self.linear(decoder_out)
    return out
  
  def generate(self, x, sos_idx, eos_idex, max_new_tokens=128):
    # x: (1, T) - source article input

    # Encode the input
    embed_x = self.input_embed(x)
    pos_embed_x = self.input_pos_encoder(x)
    embed_input = embed_x + pos_embed_x
    embed_input = self.input_dropout(embed_input)
    for layer in self.encoder:
      embed_input = layer(embed_input)
    encoder_out = embed_input

    # Start with <sos> token
    y = torch.full((1, 1), sos_idx, dtype=torch.long, device=x.device)

    for _ in range(max_new_tokens):
      embed_y = self.target_embed(y)
      pos_embed_y = self.target_pos_encoder(y)
      embed_target = embed_y + pos_embed_y
      embed_target = self.target_dropout(embed_target)

      for layer in self.decoder:
        embed_target = layer(embed_target, encoder_out)

      logits = self.linear(embed_target)
      next_token_logits = logits[:, -1, :]  # (B, vocab_size)
      probs = torch.softmax(next_token_logits, dim=-1)
      next_token = torch.multinomial(probs, num_samples=1)  # (B, 1)
      y = torch.cat([y, next_token], dim=1)
      if next_token.item() == eos_idex:
        break

    return y


## Env

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

max_input_length = 512
max_target_length = 128

vocab_size = len(tokenizer.vocab)
pad_idx = tokenizer.special_tokens['<pad>']

num_layers = 6
num_heads = 8
embed_dim = 512
input_dropout = 0.1

lr = 0.001
epochs = 10
eval_interval = 1
batch_size = 32
batch_number_train = len(train_dataset) // batch_size
batch_number_val = len(val_dataset) // batch_size

## Training

In [ ]:
SEED = 42

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

model = Transformer_Encoder_Decoder(vocab_size, num_layers, num_heads, embed_dim, max_input_length, max_target_length, pad_idx, input_dropout).to(device)
loss_fn = nn.CrossEntropyLoss(ignore_index=pad_idx)
optimizer = optim.AdamW(model.parameters(), lr=lr)

for epoch in range(epochs):
  model.train()

  train_loss = 0

  for out_dict in generate_batches(dataset=train_dataset, batch_size=batch_size, device=device):
    x = out_dict['x']
    y = out_dict['y']

    y_logits = model(x, y[:, :-1])
    
    B, T, C = y_logits.shape
    loss = loss = loss_fn(y_logits.reshape(B * T, C), y[:, 1:].reshape(B * T))
    # print(loss)
    train_loss += loss.item()

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

  train_loss /= batch_number_train

  if epoch % eval_interval == 0 or epoch == epochs-1:
    model.eval()
    val_loss = 0

    with torch.inference_mode():
      for out_dict in generate_batches(dataset=val_dataset, batch_size=batch_size, device=device):
        x = out_dict['x']
        y = out_dict['y']

        y_logits = model(x, y[:, :-1])
    
        B, T, C = y_logits.shape
        loss = loss_fn(y_logits.reshape(B * T, C), y[:, 1:].reshape(B * T))
        val_loss += loss.item()

    val_loss /= batch_number_val
    
    print(f"Epoch {epoch} | Train loss: {train_loss:.4f} | Val loss: {val_loss:.4f}")

Epoch 0 | Train loss: 6.0024 | Val loss: 6.0043
Epoch 1 | Train loss: 5.9716 | Val loss: 5.9968
Epoch 2 | Train loss: 5.9515 | Val loss: 6.9783
Epoch 3 | Train loss: 5.9316 | Val loss: 6.7167
Epoch 4 | Train loss: 5.9271 | Val loss: 6.8649
Epoch 5 | Train loss: 5.9247 | Val loss: 6.5532
Epoch 6 | Train loss: 5.9230 | Val loss: 6.9263
Epoch 7 | Train loss: 5.9216 | Val loss: 7.1859
Epoch 8 | Train loss: 5.9205 | Val loss: 6.3286
Epoch 9 | Train loss: 5.9204 | Val loss: 6.6835


In [ ]:
# Save model weights and checkpoint

torch.save(model.state_dict(), 'model/model_weights.pth')

torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
}, 'model/checkpoint.pth')

## Testing

In [29]:
SEED = 42

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

batch_number_test = len(test_dataset) // batch_size

model = Transformer_Encoder_Decoder(vocab_size, num_layers, num_heads, embed_dim, max_input_length, max_target_length, pad_idx, input_dropout).to(device)
model.load_state_dict(torch.load('model/model_weights.pth'))
loss_fn = nn.CrossEntropyLoss(ignore_index=pad_idx)

model.eval()
test_loss = 0

with torch.inference_mode():
  for out_dict in generate_batches(dataset=test_dataset, batch_size=batch_size, device=device):
    x = out_dict['x']
    y = out_dict['y']

    y_logits = model(x, y[:, :-1])
    
    B, T, C = y_logits.shape
    loss = loss_fn(y_logits.reshape(B * T, C), y[:, 1:].reshape(B * T))
    test_loss += loss.item()

test_loss /= batch_number_test

print(f"Test loss: {test_loss:.4f}")

Test loss: 6.6941


## Generate Summary

In [26]:
new_model = Transformer_Encoder_Decoder(vocab_size, num_layers, num_heads, embed_dim, max_input_length, max_target_length, pad_idx, input_dropout).to(device)
new_model.load_state_dict(torch.load('model/model_weights.pth'))

<All keys matched successfully>

In [27]:
input = article_vectorizer.index_vectorize(train_tokenized['article'][0], max_length=512)
input_tensor = torch.as_tensor(input).unsqueeze(0).to(device='cuda')
output_tensor = new_model.generate(input_tensor, tokenizer.special_tokens['<sos>'], tokenizer.special_tokens['<eos>'])

print("Input : ", tokenizer.decode(input_tensor.squeeze().cpu().numpy()))
print("Output: ", tokenizer.decode(output_tensor.squeeze().cpu().numpy()))

Input :  <sos>LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. 

In [28]:
input = article_vectorizer.index_vectorize(test_tokenized['article'][0], max_length=512)
input_tensor = torch.as_tensor(input).unsqueeze(0).to(device='cuda')
output_tensor = new_model.generate(input_tensor, tokenizer.special_tokens['<sos>'], tokenizer.special_tokens['<eos>'])

print("Input : ", tokenizer.decode(input_tensor.squeeze().cpu().numpy()))
print("Output: ", tokenizer.decode(output_tensor.squeeze().cpu().numpy()))

Input :  <sos>(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday'